In [ ]:

!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 5.5 MB/s 


In [ ]:
!pip install haversine
import haversine as hs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from itertools import product
from math import sqrt
import pandas as pd
from pandas import DataFrame
from gurobipy import GRB
import os
from geopy.distance import geodesic

In [ ]:
def compute_distance(loc1, loc2):
   
    valor=geodesic(loc1, loc2).kilometers    
    return valor

In [ ]:
def solve_sjl_logisticaHumanitaria_distancias(c_coordinates, demand,m,n): # recibe 
    #####################################################
    #                    Data
    #####################################################
    # Indices for the counties
    counties = [*range(1,m+1)]
        
    # Indices for the facilities
    facilities = [*range(1,n+3)]
        
    #########################################################
    
    #########################################################
    
    df=pd.read_excel('geolocalizacionAsignacion7Centros.xlsx')#, index_col=5)  
    dicc=list()
    for x in range(n):
        b5=[df.iloc[x]['lat'],df.iloc[x]['lng']]
        b5=tuple(b5)
        dicc.append(b5)
    
    k=n+1
    numbers = range(1, k)
    secuencia = [number for number in numbers]
    
    zipbObj = zip(secuencia, dicc)
    
    diccionarioA = dict(zipbObj)
    
    from collections import defaultdict
    d1 = diccionarioA
    
    dicc2=list()
    for x in range(n):
        b=df.iloc[x]['kits']
        dicc2.append(b)  
    
    zipbObj2 = zip(secuencia, dicc2)
    
    diccionarioB = dict(zipbObj2)
    d2 = diccionarioB  
    dd = defaultdict(list)
    for d in (d1, d2): 
        for key, value in d.items():
            dd[key].append(value)
    
    existing, e_coordinates, e_capacity  = gp.multidict(dd)
    
    temporary, t_coordinates, t_capacity  = gp.multidict({
        n+1: [(-12.0194141,-76.9526184), 15],
        n+2: [(-12.0186654,-76.9522931), 20]
    })
   
    # Compute key parameters of MIP model formulation
    f_coordinates = {}
    for e in existing:
        f_coordinates[e] = e_coordinates[e]
    for t in temporary:
        f_coordinates[t] = t_coordinates[t]
    
    cf = []
    for c in counties:
        for f in facilities:
            tp = c,f
            cf.append(tp)
    # Compute distances between counties centroids and facility locations
    distance = {(c,f): compute_distance(c_coordinates[c], f_coordinates[f]) for c, f in cf}
    df = pd.DataFrame(data=distance, index=[0])
    df = (df.T)  
    df.to_excel('distancias.xlsx', sheet_name='sheet1')
    df=pd.read_excel('distancias.xlsx') 
    nombres=["sector","centro","distancia"]
    df.columns= nombres
    #### Obtenemos el total de las distancias recorridas con "n" centros de ayuda
    lista1= df.loc[:,["centro","distancia"]]
    lista1=lista1[lista1["centro"]< n+1]
    df_pivot=pd.pivot_table(data=lista1,values="distancia",aggfunc="sum",columns= "centro", margins = False, margins_name='Total').T
    df2 = df_pivot.reset_index()
    total=df2["distancia"].sum()

    df_pivot2=pd.pivot_table(data=lista1,values="distancia",aggfunc="mean",columns= "centro", margins = False, margins_name='Total').T
    df3 = df_pivot2.reset_index()
    total2=df3["distancia"].mean()
    return total , total2

    

In [ ]:
def solve_sjl_logisticaHumanitaria(c_coordinates, demand,sectores,n):
    #####################################################
    #                    Data
    #####################################################
    # Indices - sectores
    counties = [*range(1,sectores+1)]
        
    # Indices facilities
    facilities = [*range(1,n+3)]
        
    #########################################################
    
    #########################################################
    
    df=pd.read_excel('geolocalizacionAsignacion7Centros.xlsx')
    dicc=list()
    for x in range(n):
        b5=[df.iloc[x]['lat'],df.iloc[x]['lng']]
        b5=tuple(b5)
        dicc.append(b5)
    
    k=n+1
    numbers = range(1, k)
    secuencia = [number for number in numbers]
    
    zipbObj = zip(secuencia, dicc)
    diccionarioA = dict(zipbObj)
    
    from collections import defaultdict
    d1 = diccionarioA

    dicc2=list()
    for x in range(n):
        b=df.iloc[x]['kits']
        dicc2.append(b)  
    # Create a zip object from two lists
    zipbObj2 = zip(secuencia, dicc2)
    # Create a dictionary from zip object
    diccionarioB = dict(zipbObj2)
    d2 = diccionarioB  
    dd = defaultdict(list)
    for d in (d1, d2): 
        for key, value in d.items():
            dd[key].append(value)
    
    existing, e_coordinates, e_capacity  = gp.multidict(dd)
    
    temporary, t_coordinates, t_capacity  = gp.multidict({
        n+1: [(-12.0194141,-76.9526184), 15],
        n+2: [(-12.0186654,-76.9522931), 20]
    })
    # Costs
    dcost = 50
    # Cost of building a temporary facility
    tfcost = 500000
    # Compute key parameters of MIP model formulation
    f_coordinates = {}
    for e in existing:
        f_coordinates[e] = e_coordinates[e]
    for t in temporary:
        f_coordinates[t] = t_coordinates[t]
    
    cf = []
    for c in counties:
        for f in facilities:
            tp = c,f
            cf.append(tp)
    # Compute distances 
    distance = {(c,f): compute_distance(c_coordinates[c], f_coordinates[f]) for c, f in cf} 

    #####################################################
    #                    MIP Model Formulation
    #####################################################
    m = gp.Model('logisticaHumanitariaSJL')
    # Build temporary facility
    y = m.addVars(temporary, vtype=GRB.BINARY, name='temporary')
    # 
    x = m.addVars(cf, vtype=GRB.CONTINUOUS, name='Assign')
    # Add capacity to temporary facilities
    z = m.addVars(temporary, vtype=GRB.CONTINUOUS, name='addCap' )
    # Objective function: Minimize total distance to Aid facility
    # Penalty for adding capacity at a temporary facility
    bigM = 1e9
    m.setObjective(gp.quicksum(dcost*distance[c,f]*x[c,f] for c,f in cf) 
                   + tfcost*y.sum()
                   + bigM*z.sum(), GRB.MINIMIZE)
    # Demand constraints
    demandConstrs = m.addConstrs((gp.quicksum(x[c,f] for f in facilities) == demand[c] for c in counties), 
                                 name='demandConstrs')
    # Existing facilities capacity constraints
    existingCapConstrs = m.addConstrs((gp.quicksum(x[c,e]  for c in counties) <= e_capacity[e] for e in existing ), 
                                      name='existingCapConstrs')
    # temporary facilities capacity constraints
    temporaryCapConstrs = m.addConstrs((gp.quicksum(x[c,t]  for c in counties) -z[t] 
                                        <= t_capacity[t]*y[t] for t in temporary ),
                                       name='temporaryCapConstrs')
    # Run optimization engine
    m.optimize()
    #####################################################
    #                    Output Reports
    #####################################################
    # Total cost of building temporary facility locations
    temporary_facility_cost = 0
    #print(f"\n\n_____________Optimal costs______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            temporary_facility_cost += tfcost*round(y[t].x)
    patient_allocation_cost = 0
    for c,f in cf:
        if x[c,f].x > 1e-6:
            patient_allocation_cost += dcost*round(distance[c,f]*x[c,f].x)
    #print(f"El costo total de implementar centros de respuesta temporal ante la emergencia es ${temporary_facility_cost:,}") 
    #print(f"El costo total de implementar en los centros ya establecidos para ese fin es ${patient_allocation_cost:,}")  
    # Build temporary facility at location
    #print(f"\n_____________Plan para centros de respuesta temporal______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            1
            #print(f"Se requiere implementar un centro temporal de atención en {t}")
    # Extra capacity at temporary facilities
    #print(f"\n_____________Plan para incrementar la capacidad en centros de respuesta temporales______________________")
    for t in temporary:
        if (z[t].x > 1e-6):
            1
            #print(f"Incrementar la capacidad del centro de atencion temporal en la ubicacion {t} con {round(z[t].x)} kits de ayuda")
    # Demand satisfied at each facility
    f_demand = {}
   # print(f"\n_____________Asignación de los danmificados  en el centro de atención ______________________")
    
    
    for f in facilities:
        temp = 0
        for c in counties:
            allocation = round(x[c,f].x)
            if allocation > 0:
                allocation
                print(f"{allocation} Los danmificados del sector {c} deben ser asignados al centro de apoyo  {f} ")
                              
               
            temp += allocation
            
            f_demand[f] = temp
           # print(f"{temp} es el total de danmificados atendidos en el centro de apoyo {f}. ")
           # print(f"\n________________________________________________________________________________")   
    
    ####  PERMITE SABER  SI SE LOGRA CUBRIR LA NECESIDAD DEL SECTOR 
    ####  DETALLA PARA CADA SECTOR DE QUE CENTRO DE AYUDA PROVIENE EL KIT


    cf2 = []
    for c in counties:
        for f in facilities:
            tp = c,f #<--------------------------------------------------
            cf2.append(tp)
            allocation=round(x[c,f].x)
            if allocation > 0:
              asigna2 = {(c,f): round(x[c,f].x) for c,f in cf2} 
    dfasigna = pd.DataFrame(data=asigna2, index=[0])
    dfasigna = (dfasigna.T)
    dfasigna.to_excel('servicioDemandaSector.xlsx', sheet_name='sheet1')
    df=pd.read_excel('servicioDemandaSector.xlsx')
    nombres=["sector","centro","demanda"]
    df.columns= nombres

    #### Se analiza cada sector para verificar si se pudo atender
    lista1= df.loc[:,["centro","demanda"]]
    lista1=lista1[lista1["centro"]<n+1]
    sectorAtendido=[]
    sectorSinAtender=[]
    valor =0
    for i in range(0,len(lista1)):    
        valor+=lista1.iloc[i,1]
        if lista1.iloc[i,0]%n ==0 :    
            if valor ==0:
                
                sectorSinAtender.append(1)
                valor=0
            else:
                sectorAtendido.append(1)
                valor=0

          
    servidos = len(sectorAtendido) 
    porcentaje = servidos/sectores*100
    
    ######  PERMITE SABER DE DONDE PARTE LA AYUDA A QUE SECTOR
    cf3 = []
    
    for f in facilities:
        for c in counties:
            tp = f,c #<--------------------------------------------------
            cf3.append(tp)
            allocation2=round(x[c,f].x)
            if allocation2 > 0:
              asigna3 = {(f,c): round(x[c,f].x) for f,c in cf3} 
    dfasigna2 = pd.DataFrame(data=asigna3, index=[0])
    dfasigna2 = (dfasigna2.T)
    #dfasigna2.to_excel('capacidadUtilizadaCentro.xlsx', sheet_name='sheet1')
    


    # Test total demand = total demand satisfied by facilities
    total_demand = 0
    for c in counties:
        total_demand += demand[c]
    demand_satisfied = 0
    for f in facilities:
        demand_satisfied += f_demand[f] 

    nivel_servicio = demand_satisfied/total_demand*100
    print(f"\n_____________Balance requerimiento = centros atención______________________")
    print(f"El total de la demanda de kits requerido de ayuda es: {total_demand:,} kits")
    #print(f"El total de la demanda de kits de ayuda entregada es: {demand_satisfied:,} kits")
    print(f"La proporción de la demanda de kits atendida entregada es: {nivel_servicio:,} %")

    return servidos, porcentaje


In [ ]:
def crea_demanda(m,n):
    # m : es la cantidad sectores donde se debe ayudar
    
    df=pd.read_excel('demandaGrupoAsignacion3.xlsx')
    dicc=list()
    for x in range(m):
        b5=[df.iloc[x]['Latitud'],df.iloc[x]['Longitud']]
        b5=tuple(b5)
        dicc.append(b5)
    dicc
    k=m+1
    numbers = range(1, k)
    secuencia = [number for number in numbers]
    
    zipbObj = zip(secuencia, dicc)
    # Create a dictionary from zip object
    diccionarioA = dict(zipbObj)
    #diccionarioA
    from collections import defaultdict
    d1 = diccionarioA
   
    dicc2=list()
    for x in range(m):
        b=df.iloc[x]['requerimiento']
        dicc2.append(b)  
    # Create a zip object from two lists
    zipbObj2 = zip(secuencia, dicc2)
    # Create a dictionary from zip object
    diccionarioB = dict(zipbObj2)
    d2 = diccionarioB
    dd = defaultdict(list)
    for d in (d1, d2): 
        for key, value in d.items():
            dd[key].append(value)
    counties, coordinates, forecast  = gp.multidict(dd)
    sectores_servidos, porcentaje_cubierto= solve_sjl_logisticaHumanitaria(coordinates, forecast,m,n)
    return sectores_servidos, porcentaje_cubierto

In [ ]:
def crea_matrizDistancias(m,n):
    # m : es la cantidad sectores donde se debe ayudar
    
    df=pd.read_excel('demandaGrupoAsignacion3.xlsx')#, index_col=5)  
    dicc=list()
    for x in range(m):
        b5=[df.iloc[x]['Latitud'],df.iloc[x]['Longitud']]
        b5=tuple(b5)
        dicc.append(b5)
    dicc
    k=m+1
    numbers = range(1, k)
    secuencia = [number for number in numbers]
    zipbObj = zip(secuencia, dicc)
    # Create a dictionary from zip object
    diccionarioA = dict(zipbObj)

    from collections import defaultdict
    d1 = diccionarioA
    dicc2=list()
    for x in range(m):
        b=df.iloc[x]['requerimiento']
        dicc2.append(b)  
    # Create a zip object from two lists
    zipbObj2 = zip(secuencia, dicc2)
    # Create a dictionary from zip object
    diccionarioB = dict(zipbObj2)
    d2 = diccionarioB
    dd = defaultdict(list)
    for d in (d1, d2): # you can list as many input dicts as you want here
        for key, value in d.items():
            dd[key].append(value)
    counties, coordinates, forecast  = gp.multidict(dd)
    valor1,valor2 = solve_sjl_logisticaHumanitaria_distancias(coordinates, forecast,m,n)
    return valor1,valor2

# **Análisis de resultados** del modelo de Optimización y asignación de ayuda

In [ ]:
df = pd.DataFrame(columns = [30, 40, 50], 
                   index = [3, 4, 5])
list1=[30,40,50]
list2=[3,4,5]
for i in list1:
    for j in list2:
        servidos, porcentaje = crea_demanda(i,j)
        df.loc[j,i] = ("{:.0f} sectores".format(servidos),"{:.1f}%".format(porcentaje))

print("##########################")
print("##########################")
print("###### TABLA 1 ###########")
print("##########################")
print("##########################")
print("Resumen de cantidad de sectores atendidos y centros de asistencia")
df

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 35 rows, 154 columns and 304 nonzeros
Model fingerprint: 0xe54c49b4
Variable types: 152 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 5e+02]
Presolve time: 0.00s
Presolved: 35 rows, 154 columns, 304 nonzeros
Variable types: 152 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1.208001e+12

Root relaxation: cutoff, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1.2080e+12 1.2080e+12  0.00%     -    0s

Explored 1 nodes (21 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 2 (

,30,40,50
3,"(17 sectores, 56.7%)","(17 sectores, 42.5%)","(17 sectores, 34.0%)"
4,"(23 sectores, 76.7%)","(23 sectores, 57.5%)","(23 sectores, 46.0%)"
5,"(26 sectores, 86.7%)","(27 sectores, 67.5%)","(31 sectores, 62.0%)"


In [ ]:

df = pd.DataFrame(columns = [30, 40, 50], 
                   index = [3, 4, 5])
list1=[30,40,50]
list2=[3,4,5]
for i in list1:
    for j in list2:
        distancia, media = crea_matrizDistancias(i,j)
        df.loc[j,i] = ("{:.1f} km".format(distancia),"{:.1f} km".format(media))


print("##########################")
print("##########################")
print("###### TABLA 2 ###########")
print("##########################")
print("##########################")
print("Resumen de tuplas con distancias y medias entre sectores y centros de asistencia")
df


##########################
##########################
###### TABLA 2 ###########
##########################
##########################
Resumen de tuplas con distancias y medias entre sectores y centros de asistencia


,30,40,50
3,"(52.8 km, 0.6 km)","(69.5 km, 0.6 km)","(85.8 km, 0.6 km)"
4,"(67.5 km, 0.6 km)","(88.9 km, 0.6 km)","(109.1 km, 0.5 km)"
5,"(80.2 km, 0.5 km)","(105.4 km, 0.5 km)","(128.8 km, 0.5 km)"
